In [1]:
import librosa
import soundfile
import pyaudio
import glob
import os
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
#read the data file 
#3 features below:
    #mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
    #chroma: Pertains to the 12 different pitch classes
    #mel: Mel Spectrogram Frequency

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X,sample_rate=librosa.load(file_name)
        #X = sound_file.read(dtype='float32')
        #sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [4]:
#create the dictionary with numbers and emotions

In [5]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [6]:
#load the data and extract features(x,y) for each sound file data 

In [7]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('/Users/aya/Desktop/speech_emotion_analyzer/AyaData/Audio_Speech_Actors_01-24/Actor_*/*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
#split the data into training and testing sets (test:25%)

In [9]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [65]:
model=MLPClassifier(alpha=0.01, batch_size=128, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [66]:
#train the model

In [67]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=128, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [68]:
#predict the test set

In [69]:
y_pred=model.predict(x_test)

In [70]:
#accuracy of our model

In [71]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [72]:
#Print the accuracy

In [73]:
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 75.00%
